In [13]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings



In [28]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [30]:
extracted_data = load_pdf_file(data ='D:\projects\Medichat\Data/')

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\deepanshu\AppData\Local\Temp\ipykernel_6188\1963676096.py:1: SyntaxWarning: invalid escape sequence '\p'
  extracted_data = load_pdf_file(data ='D:\projects\Medichat\Data/')


In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunks = text_splitter.split_documents(extracted_data)
    return texts_chunks

In [31]:
text_chunks = text_split(extracted_data)
print("length", len(text_chunks))

length 8624


In [32]:
from transformers import AutoModel, AutoTokenizer

model_name = "sentence-transformers/all-MiniLM-L6-v2"

# This will now use Xet Storage (if supported by the model repo)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [33]:
pip install huggingface_hub[hf_xet]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_huggingface_embedding()


C:\Users\deepanshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

length 384


In [3]:
#query_result

In [75]:
from dotenv import load_dotenv
load_dotenv()

True

In [76]:
PINECONE_API_KEY= os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY= os.environ.get("OPENAI_API_KEY")

In [59]:
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Load API keys from environment variables
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Only if you'll use OpenAI later

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Index name
index_name = "medichat"

# Create index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# Connect to the index
index = pc.Index(index_name)
index



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '853330ca653ef5823d75a7e2f9ccf05c', 'date': 'Wed, 06 Aug 2025 10:43:55 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [77]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [54]:
# pip install langchain-pinecone pinecone-client

In [78]:
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings  # or any other embedding

# Step 1: Create embeddings for each chunk
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # or your preferred model

# Step 2: Upsert the embeddings into Pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=text_chunks,             # your list of Document objects
    embedding=embeddings,             # embedding object
    index_name="medichat"             # your Pinecone index name
)


KeyboardInterrupt: 

In [61]:
vectorstore

In [81]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [79]:
from langchain_openai import OpenAI 
llm = OpenAI(temperature=0.4, max_tokens=500)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate 

system_prompt = (
    "You are Medichat, a helpful, trustworthy, and knowledgeable AI assistant trained to answer medical questions based on information retrieved from verified medical documents. Use only the provided context to generate responses. If a user asks something outside the context or your knowledge, politely say you don't have enough information. Never guess, speculate, or give personal medical advice. Do not provide diagnoses or prescriptions. Always remind users to consult a licensed medical professional for serious issues. Keep your responses clear, factual, and easy to understand."

)

prompt = ChatPromptTemplate.from_chat_history([
    ("system", system_prompt),
    ("human", "{input}")
])



In [86]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "input"],
    template="""
You are a knowledgeable and helpful medical assistant named MediChat. You answer user queries based on the provided context.

Context:
{context}

Question:
{input}

Answer the question using the above context. Be concise, accurate, and helpful. If the answer is not found in the context, say "I'm sorry, I could not find that information in the documents."
"""
)


In [87]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
reg_chain = create_retrieval_chain(retriever, question_answer_chain)

In [89]:
response = reg_chain.invoke({"input": "what is skin"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}